In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

# Check PyTorch Version
torch.__version__

In [ ]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
# Create some data using the linear regression formula of y = weight * X + bias
weight = .2
bias = .8

# Create range values
start = 0
end = 1
step = .02

# Create X and y (features and labels)
X = torch.arange(start, end, step).unsqueeze(dim=1) # without unsqueeze, errors will pop up
y = weight * X + bias
X[:10], y[:10]

In [ ]:
# Split data
train_split = int(.8 * len(X)) # Splitting data into 80% for training and 20% for testing
X_train = X[:train_split] #Defining that X training starts from 0 to 80% of data
X_test = X[train_split:] # Defining that the remaining data is for testing
y_train = y[:train_split]
y_test = y[train_split:]
len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
# Graph Function
def plot_predictions(train_data=X_train,
                     train_labels=y_train,
                     test_data=X_test,
                     test_labels=y_test,
                     predictions=None):
  # Plots training data, test data and compares predictions
  plt.figure(figsize=(10,7))

  #Plot training data in blue
  plt.scatter(train_data, train_labels, c="b", s=4, label="Training data")

  #Plot test data in green
  plt.scatter(test_data, test_labels, c="g", s=4, label="Testing data")

  # Are there predictions?
  if predictions is not None:
    # Plot the predictions if they exist
    plt.scatter(test_data, predictions, c="r", s=4, label="Predictions")

    # Show the legend
    plt.legend(prop={'size': 20});

In [ ]:
# VISUALIZE
plot_predictions(X_train, y_train, X_test, y_test)

In [ ]:
# Building a PyTorch Linear Model
# Create a linear model by subclassing nn.Module
class LinearRegressionModelV2(nn.Module):
  def __init__(self):
    super().__init__()
    # Use nn.Linear() for creating the model parameters / also called: linear transform
    self.linear_layer = nn.Linear(in_features=1,   # Little shortcut to prev class code, using linear layer. The input is 1 x gets 1 y so the input=1 and output=1
                                  out_features=1)

  def forward(self, x: torch.Tensor) -> torch.Tensor:
    return self.linear_layer(x)

# Set manual seed
torch.manual_seed(87)
model_1 = LinearRegressionModelV2() # Create an instance of the model
model_1, model_1.state_dict() # Call state_dict() on the instance

In [ ]:
# Check the model current device
next(model_1.parameters()).device

In [ ]:
# Set the model to use the target device
model_1.to(device)
next(model_1.parameters()).device

In [ ]:
# Setup loss function
loss_fn = nn.L1Loss() # same as MAE

# Setup our optimizer
optimizer = torch.optim.SGD(params=model_1.parameters(),
                            lr=0.01)

In [ ]:
#Training loop
torch.manual_seed(87)

epochs = 1000

# Put data on the target device (device agnostic code for data)
# X_train = X_train.to(device) # This line is not needed here
# X_test = X_test.to(device) # This line is not needed here
# y_train = y_train.to(device) # This line is not needed here
# y_test = y_test.to(device) # This line is not needed here


for epoch in range(epochs):
  model_1.train()

  # 1. Forward Pass
  y_pred = model_1(X_train.to(device)) # Move X_train to the device here

  # 2. Calculate the loss
  loss = loss_fn(y_pred, y_train.to(device)) # Move y_train to the device here

  # 3. Optimizer zero grad
  optimizer.zero_grad()

  # 4. Backpropagation
  loss.backward()

  # 5. Optimizer step
  optimizer.step()

  ### Testing
  model_1.eval()
  with torch.inference_mode():
    test_pred = model_1(X_test.to(device)) # Move X_test to the device here

    test_loss = loss_fn(test_pred, y_test.to(device)) # Move y_test to the device here

  # Print out whats happenin

  if epoch % 10 ==0:
    print(f"Epoch: {epoch} | Loss: {loss} | Test loss: {test_loss}")

In [ ]:
model_1.state_dict()

In [ ]:
# Turn model into eval mode
model_1.eval()

# Make predictions on the test data
with torch.inference_mode():
  y_preds = model_1(X_test)
y_preds

In [ ]:
# Check out our model predictions visually
plot_predictions(predictions = y_preds.cpu())